### Importing all the relevant libraries

In [23]:
import matplotlib.pyplot as plt
import pkg_resources
import pandas as pd
import numpy as np
import seaborn as sns
import sobol_seq
import time
import types

def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for rq in requirements:
    print("{}=={}".format(*r))

seaborn==0.9.0
pandas==0.24.2
numpy==1.14.5
matplotlib==2.2.2


import matplotlib
import matplotlib.pyplot as plt
import pkg_resources
import pandas as pd
import numpy as np
import seaborn
import sobol_seq
import time
import types

def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

In [2]:
df = pd.read_excel('Database_Final_UPD(3).xlsx')

### Generate the series of quasi-random numbers for the analysis

In [3]:
quasiRandom_df = pd.DataFrame(sobol_seq.i4_sobol_generate(12,8192))

DistributionFiMax = 0.8+quasiRandom_df[0]*0.2
DistributionFiMin = 0.2+quasiRandom_df[1]*0.2
DistributionFiMaxB = 0.8+quasiRandom_df[6]*0.2
DistributionFiMinB = 0.2+quasiRandom_df[7]*0.2

Exchange_selector = quasiRandom_df[2].round(0).astype(int)
Exchange_selectorB = quasiRandom_df[8].round(0).astype(int)

years_residue = (quasiRandom_df[3]*(2016-2007)+1).astype(int)
years_residueB = (quasiRandom_df[9]*(2016-2007)+1).astype(int)

residual_approach = quasiRandom_df[4].round(0).astype(int)
residual_approachB = quasiRandom_df[10].round(0).astype(int)

### Rearrange the dataframe for our convenience

In [4]:
dfb = df[['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year','CF_TOTAL','EAGGF','ERDF_TOTAL','ESF']].copy()
df2 = dfb.melt(id_vars=['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year'],var_name='FundingScheme')
df3 = df2.pivot_table(index=['ProgrammingPeriod','FundingScheme','Country','NUTS1Code','NUTS2Code'],columns='Year', values='value')
df4 = df3.dropna(how='all').fillna(0)

### Get the slice corresponding to the set we'll be working on

In [5]:
df4b = df4.loc['2007-2013','ERDF_TOTAL',:,:,:].droplevel([0,1])

df4c = df4.loc['2007-2013','CF_TOTAL',:,:,:].droplevel([0,1])

df4d = df4b+df4c

### Excluding years before the commencing of the programming periods

In [6]:
dummy = []

diff = int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][5:])+6-\
int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][:4])

d_Var = pd.Series([k/diff for k in range(1,diff+1)],[y for y in \
range(int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][:4]),
int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][5:])+6)])

dummy.append(d_Var)

for pp in np.sort(np.array(list(set(df.ProgrammingPeriod))))[1:]:
    diff = int(pp[5:])+4-int(pp[:4])
    
    d_Var = pd.Series([k/diff for k in range(1,diff+1)],[y for y in range(int(pp[:4]),int(pp[5:])+4)])
    
    dummy.append(d_Var)

### Normalise the cumulative figures

In [7]:
df5 = df4d.copy()

Norm_df6 = ((df5.cumsum(axis=1).T/df5.cumsum(axis=1).max(axis=1).values).T).dropna(how='all')

### Define the outcome variables

In [8]:
Norm_df6['$\delta$']=(Norm_df6.iloc[:,-10:]-dummy[-1]).cumsum(axis=1).iloc[:,-1]

Norm_df6['$\mu$']=(Norm_df6['$\delta$'].max()-Norm_df6['$\delta$'])/(Norm_df6['$\delta$'].max()-Norm_df6['$\delta$'].min())

### Set the trigger for the number of years the residual expenditure gets spread onto on the last eligible year of the programming perido

In [9]:
ld1 = []
for iq,qr in enumerate(quasiRandom_df[5]):
    df8b = Norm_df6.copy()
    df8b[0]=(qr*(Norm_df6['$\mu$']*(len(dummy[-1])-1)).astype(int)).astype(int)+1
    for il in range(1,len(dummy[-1])):
        df8b[il]=df8b[0]-il
    df8b[df8b<1]=1
    cd = [il0 for il0 in range(len(dummy[-1]))]
    df8b['value']=iq
    cd.append('value')
    df8b = df8b[cd]
    ld1.append(df8b)
years = pd.concat(ld1)
years.set_index('value', append=True, inplace=True)

ld1b = []
for iq,qr in enumerate(quasiRandom_df[11]):
    df8b = Norm_df6.copy()
    df8b[0]=(qr*(Norm_df6['$\mu$']*(len(dummy[-1])-1)).astype(int)).astype(int)+1
    for il in range(1,len(dummy[-1])):
        df8b[il]=df8b[0]-il
    df8b[df8b<1]=1
    cd = [il0 for il0 in range(len(dummy[-1]))]
    df8b['value']=iq
    cd.append('value')
    df8b = df8b[cd]
    ld1b.append(df8b)
yearsb = pd.concat(ld1b)
yearsb.set_index('value', append=True, inplace=True)

### Define the yearly residues

In [10]:
def A(n):
    return [(2**j)/(2**n-1) for j in reversed(range(n))]

B9 = []
for k in reversed(range(2,11)):
    B9.append(pd.DataFrame([A(y) for y in range(1,k)],index=[y for y in range(1,k)],
                           columns=[y for y in range(1,k)]).fillna(0).sort_values(by=1,ascending=False))


In [11]:
def C(n):
    return [1/n for j in reversed(range(n))]

C9 = pd.DataFrame([C(y) for y in range(1,11)],index=[y for y in range(1,11)],
                           columns=[y for y in range(1,11)]).fillna(0).sort_values(by=1,ascending=False)

D = [C9,pd.DataFrame([A(y) for y in range(1,11)],index=[y for y in range(1,11)],
                           columns=[y for y in range(1,11)]).fillna(0).sort_values(by=1,ascending=False)]

In [12]:
Payments = df4b.loc['CZ'].iloc[:,-10:].copy().droplevel(0)

### Prepare the member-state database and the set of exchange rates

In [13]:
df1 = pd.read_excel('nuts_prog_kat_Cohesion_codesonly_v3.xlsx',usecols=[0,1,5,6,9,10,11,12])
df_REGIO = df1[(df1['CCI'].str.contains("161"))|df1['CCI'].str.contains("162")]
df_REGIO_capped = df_REGIO[df_REGIO.year<2017]
ER = pd.read_csv('CZK_EURO_historical_exchange_rate.csv')
ExchangeRates = ER.copy()
ExchangeRates['year']=ER.date.astype(str).str[:4].astype(int)
ExchangeRates = ExchangeRates[(ExchangeRates['year']>2006)&(ExchangeRates['year']<2017)]
ExchangeRates=ExchangeRates.drop(['date','conf'],axis=1).set_index('year')
df_REGIO_yearly = df_REGIO_capped.groupby('year')
df_REGIO_su = pd.DataFrame([dfr['EU (czk)'].sum() for idf, dfr in df_REGIO_yearly], 
                            index=[idf for idf, dfr in df_REGIO_yearly],columns=['sum'])

ExchangeRates[1]=ExchangeRates['s1'].mean()
exchange_rate = ExchangeRates.T
exchange_rate.index = [0,1]


df_REGIO_REGIO_yearly=df_REGIO_capped.groupby(['year','NUTS2']).sum()
df_REGIO_pv = df_REGIO_REGIO_yearly.pivot_table(index='year', columns='NUTS2', values='EU (czk)').fillna(0).T

### Evaluate the distance between the modelled expenditure and the MS incurred

In [14]:
Expe = pd.concat([Payments.copy() for r in range(len(quasiRandom_df))])

Exp = Expe.copy()
ExpAB1 = Expe.copy()
ExpAB2 = Expe.copy()
ExpAB6 = Expe.copy()
ExpB = Expe.copy()

Exp['r']=np.array([r for r in range(len(quasiRandom_df)) for er in range(len(Payments))])
N_df = pd.concat([Norm_df6.loc['CZ','$\mu$'] for r in range(len(quasiRandom_df))]).droplevel(0)

ExpAB1['r']=Exp['r']
ExpAB2['r']=Exp['r']
ExpAB6['r']=Exp['r']
ExpB['r']=Exp['r']

Est_expenditure = pd.concat([df_REGIO_pv.copy() for r in range(len(quasiRandom_df))])

y_slice = years.loc[pd.IndexSlice['CZ',:,:]].droplevel(0)

y_sliceB = yearsb.loc[pd.IndexSlice['CZ',:,:]].droplevel(0)

Exp.loc[:,2016]=Expe.loc[:,2016]*(DistributionFiMax[Exp.r].values-N_df*(DistributionFiMax[Exp.r]-\
                                                                        DistributionFiMin[Exp.r]).values)
ExpAB1.loc[:,2016]=Expe.loc[:,2016]*(DistributionFiMax[Exp.r].values-N_df*(DistributionFiMax[Exp.r]-\
                                                                        DistributionFiMinB[Exp.r]).values)
ExpAB2.loc[:,2016]=Expe.loc[:,2016]*(DistributionFiMaxB[Exp.r].values-N_df*(DistributionFiMaxB[Exp.r]-\
                                                                        DistributionFiMin[Exp.r]).values)
ExpAB6.loc[:,2016]=Expe.loc[:,2016]*(DistributionFiMax[Exp.r].values-N_df*(DistributionFiMax[Exp.r]-\
                                                                        DistributionFiMin[Exp.r]).values)
ExpB.loc[:,2016]=Expe.loc[:,2016]*(DistributionFiMaxB[Exp.r].values-N_df*(DistributionFiMaxB[Exp.r]-\
                                                                        DistributionFiMinB[Exp.r]).values)

D_df = pd.concat([D[row].iloc[years_residue[ir]] for ir, row in residual_approach[Exp.r].iteritems()],axis=1).T.values
D_dfAB4 = pd.concat([D[row].iloc[years_residueB[ir]] for ir, row in residual_approach[Exp.r].iteritems()],axis=1).T.values
D_dfAB5 = pd.concat([D[row].iloc[years_residue[ir]] for ir, row in residual_approachB[Exp.r].iteritems()],axis=1).T.values
D_dfB = pd.concat([D[row].iloc[years_residueB[ir]] for ir, row in residual_approachB[Exp.r].iteritems()],axis=1).T.values

D_exchange = pd.concat([exchange_rate.iloc[ir] for ir in Exchange_selector for er in range(len(Payments))],axis=1).T.values
D_exchangeB = pd.concat([exchange_rate.iloc[ir] for ir in Exchange_selectorB for er in range(len(Payments))],axis=1).T.values

for iy2,y2 in enumerate(reversed(range(2007,Payments.columns.max()))):
    Exp[y2] = Expe[y2]*(DistributionFiMax[Exp.r].values-N_df*(DistributionFiMax[Exp.r]-DistributionFiMin[Exp.r]).values)
    ExpAB1[y2] = Expe[y2]*(DistributionFiMax[Exp.r].values-N_df*(DistributionFiMax[Exp.r]-DistributionFiMinB[Exp.r]).values)
    ExpAB2[y2] = Expe[y2]*(DistributionFiMaxB[Exp.r].values-N_df*(DistributionFiMaxB[Exp.r]-DistributionFiMin[Exp.r]).values)
    ExpAB6[y2] = Expe[y2]*(DistributionFiMax[Exp.r].values-N_df*(DistributionFiMax[Exp.r]-DistributionFiMin[Exp.r]).values)
    ExpB[y2] = Expe[y2]*(DistributionFiMaxB[Exp.r].values-N_df*(DistributionFiMaxB[Exp.r]-DistributionFiMinB[Exp.r]).values)
    
    for iy3,y3 in enumerate(reversed(range(y2,Payments.columns.max()))):
        Exp[y2]+=Expe[y3+1]*(1-DistributionFiMax[Exp.r].values+N_df*(DistributionFiMax[Exp.r]-\
                DistributionFiMin[Exp.r]).values)*B9[iy3+len(dummy[0])-\
                len(dummy[-1])].loc[y_slice.iloc[:,iy3].values,(y3+1-y2)].values
        ExpAB1[y2]+=Expe[y3+1]*(1-DistributionFiMax[Exp.r].values+N_df*(DistributionFiMax[Exp.r]-\
                DistributionFiMinB[Exp.r]).values)*B9[iy3+len(dummy[0])-\
                len(dummy[-1])].loc[y_slice.iloc[:,iy3].values,(y3+1-y2)].values
        ExpAB2[y2]+=Expe[y3+1]*(1-DistributionFiMaxB[Exp.r].values+N_df*(DistributionFiMaxB[Exp.r]-\
                DistributionFiMin[Exp.r]).values)*B9[iy3+len(dummy[0])-\
                len(dummy[-1])].loc[y_slice.iloc[:,iy3].values,(y3+1-y2)].values
        ExpAB6[y2]+=Expe[y3+1]*(1-DistributionFiMax[Exp.r].values+N_df*(DistributionFiMax[Exp.r]-\
                DistributionFiMin[Exp.r]).values)*B9[iy3+len(dummy[0])-\
                len(dummy[-1])].loc[y_sliceB.iloc[:,iy3].values,(y3+1-y2)].values
        ExpB[y2]+=Expe[y3+1]*(1-DistributionFiMax[Exp.r].values+N_df*(DistributionFiMax[Exp.r]-\
                DistributionFiMin[Exp.r]).values)*B9[iy3+len(dummy[0])-\
                len(dummy[-1])].loc[y_sliceB.iloc[:,iy3].values,(y3+1-y2)].values

Exp[2007] += Expe[2007]*(1-DistributionFiMax[Exp.r].values+N_df*(DistributionFiMax[Exp.r]-DistributionFiMin[Exp.r]).values)
ExpAB1[2007] += Expe[2007]*(1-DistributionFiMax[Exp.r].values+N_df*(DistributionFiMax[Exp.r]-DistributionFiMinB[Exp.r]).values)
ExpAB2[2007] += Expe[2007]*(1-DistributionFiMaxB[Exp.r].values+N_df*(DistributionFiMaxB[Exp.r]-DistributionFiMin[Exp.r]).values)
ExpAB6[2007] += Expe[2007]*(1-DistributionFiMax[Exp.r].values+N_df*(DistributionFiMax[Exp.r]-DistributionFiMin[Exp.r]).values)
ExpB[2007] += Expe[2007]*(1-DistributionFiMaxB[Exp.r].values+N_df*(DistributionFiMaxB[Exp.r]-DistributionFiMinB[Exp.r]).values)

Exp = Exp.drop('r',axis=1)
ExpAB1 = ExpAB1.drop('r',axis=1)
ExpAB2 = ExpAB2.drop('r',axis=1)
ExpAB6 = ExpAB6.drop('r',axis=1)
ExpB = ExpB.drop('r',axis=1)

Excess = (Est_expenditure/D_exchange).sum(axis=1)-pd.concat([Payments.sum(axis=1) for r in range(len(quasiRandom_df))])
ExcessB = (Est_expenditure/D_exchangeB).sum(axis=1)-pd.concat([Payments.sum(axis=1) for r in range(len(quasiRandom_df))])

Residue = pd.concat([Excess for co in range(10)],axis=1).T.set_index(pd.Index([yRe for yRe in range(2007,2017)])).T
ResidueB = pd.concat([ExcessB for co in range(10)],axis=1).T.set_index(pd.Index([yRe for yRe in range(2007,2017)])).T

MS_Expenditure = (Est_expenditure/D_exchange-Residue*D_df)
MS_ExpenditureAB3 = (Est_expenditure/D_exchangeB-ResidueB*D_df)
MS_ExpenditureAB4 = (Est_expenditure/D_exchange-Residue*D_dfAB4)
MS_ExpenditureAB5 = (Est_expenditure/D_exchange-Residue*D_dfAB5)
MS_ExpenditureB = (Est_expenditure/D_exchangeB-ResidueB*D_dfB)

Distance = pd.DataFrame((np.abs(MS_Expenditure-Exp).T/Expe.sum(axis=1)).T.cumsum(axis=1).iloc[:,
                -1]).rename(columns={2016:'Distance'})
DistanceAB1 = pd.DataFrame((np.abs(MS_Expenditure-ExpAB1).T/Expe.sum(axis=1)).T.cumsum(axis=1).iloc[:,
                -1]).rename(columns={2016:'Distance'})
DistanceAB2 = pd.DataFrame((np.abs(MS_Expenditure-ExpAB2).T/Expe.sum(axis=1)).T.cumsum(axis=1).iloc[:,
                -1]).rename(columns={2016:'Distance'})
DistanceAB3 = pd.DataFrame((np.abs(MS_ExpenditureAB3-Exp).T/Expe.sum(axis=1)).T.cumsum(axis=1).iloc[:,
                -1]).rename(columns={2016:'Distance'})
DistanceAB4 = pd.DataFrame((np.abs(MS_ExpenditureAB4-Exp).T/Expe.sum(axis=1)).T.cumsum(axis=1).iloc[:,
                -1]).rename(columns={2016:'Distance'})
DistanceAB5 = pd.DataFrame((np.abs(MS_ExpenditureAB5-Exp).T/Expe.sum(axis=1)).T.cumsum(axis=1).iloc[:,
                -1]).rename(columns={2016:'Distance'})
DistanceAB6 = pd.DataFrame((np.abs(MS_Expenditure-ExpAB6).T/Expe.sum(axis=1)).T.cumsum(axis=1).iloc[:,
                -1]).rename(columns={2016:'Distance'})
DistanceB = pd.DataFrame((np.abs(MS_ExpenditureB-ExpB).T/Expe.sum(axis=1)).T.cumsum(axis=1).iloc[:,
                -1]).rename(columns={2016:'Distance'})

Distance['r']=np.array([r for r in range(len(quasiRandom_df)) for er in range(len(Payments))])
DistanceAB1['r']=Distance['r']
DistanceAB2['r']=Distance['r']
DistanceAB3['r']=Distance['r']
DistanceAB4['r']=Distance['r']
DistanceAB5['r']=Distance['r']
DistanceAB6['r']=Distance['r']
DistanceB['r']=Distance['r']

Distance_df = Distance.pivot_table(values='Distance',index='r',columns='NUTS2')
Distance_dfAB1 = DistanceAB1.pivot_table(values='Distance',index='r',columns='NUTS2')
Distance_dfAB2 = DistanceAB2.pivot_table(values='Distance',index='r',columns='NUTS2')
Distance_dfAB3 = DistanceAB3.pivot_table(values='Distance',index='r',columns='NUTS2')
Distance_dfAB4 = DistanceAB4.pivot_table(values='Distance',index='r',columns='NUTS2')
Distance_dfAB5 = DistanceAB5.pivot_table(values='Distance',index='r',columns='NUTS2')
Distance_dfAB6 = DistanceAB6.pivot_table(values='Distance',index='r',columns='NUTS2')
Distance_dfB = DistanceB.pivot_table(values='Distance',index='r',columns='NUTS2')

### Uncertainty analysis - distribution plots

In [35]:
for eg in Distance_df:
    sns.distplot(Distance_df[eg],bins=50,vertical=True,color='c')
    plt.xlabel('Relative frequency')
    plt.ylabel('Distance')
    plt.title(eg)
    plt.savefig('Uncertainty_analysis_'+str(eg)+'.png')
    plt.close()

### Let us now run the sensitivity analysis as to get out the input parameters affect the output uncertainty

In [ ]:
Distance_S = [Distance_df,Distance_dfAB1,Distance_dfAB2,Distance_dfAB3,Distance_dfAB4,Distance_dfAB5,Distance_dfAB6,
             Distance_dfB]

Distance_dfS= Distance_S[0].sample(frac=1000,replace=True)
Distance_dfS.index.name='idx'
Distance_S[-1] = Distance_S[-1].reindex(Distance_dfS.index)

Indices = []
for ds in Distance_S[1:-1]:
    Indices.append(Distance_S[-1]*(-Distance_dfS+ds.reindex(Distance_dfS.index)))
    Indices.append(0.5*(Distance_dfS-ds.reindex(Distance_dfS.index))**2)
    
Distance_dfS['r']=np.sort(np.array([r for r in range(len(quasiRandom_df)) for f in range(1000)]))
Distance_S[-1]['r']=Distance_dfS['r']
Var = pd.concat([Distance_dfS,Distance_S[-1]],sort=True).groupby('r').var().tail()

for i in Indices:
    i['r']=Distance_dfS['r']
    
Sensitivity_indices = pd.concat([i.groupby('r').mean()/Var for i in Indices],axis=1)

In [55]:
parameters = {'FiMax-model':DistributionFiMax,'FiMin-model':DistributionFiMin,'Exchange_rate':Exchange_selector,
              'Excess-years-taken-out-MS':years_residue,'Excess-share-years-taken-out-MS':residual_approach,
              'Residual-years-attributed-model':years.loc[pd.IndexSlice['CZ'],:].droplevel(0)[0]}

for c in list(set(Sensitivity_indices.columns)):
    Sensitivity_indices[c].plot.box()
    plt.xticks(range(1,13),list([q+'_'+p for p in parameters.keys() for q in ['S','T']]),rotation='vertical')
    plt.title(c)
    plt.savefig('Sensitivity_indices_'+str(c)+'.png',bbox_inches='tight')
    plt.close()

### Scatter plots for visual inspection of the trends

In [41]:
for eg in Distance_df:
    plt.scatter(years.loc[pd.IndexSlice['CZ'],:].droplevel(0)[0][eg],Distance_df[eg],s=0.01)
    plt.xlabel('Residual-years-attributed-model')
    plt.ylabel('Distance')
    plt.xticks(np.arange(1,years.loc[pd.IndexSlice['CZ'],:].droplevel(0)[0][eg].max()+1))
    plt.title(eg)
    plt.savefig('Scatter_plot_Residual-years-attributed-model_'+str(eg)+'.png')
    plt.close()

### Let us know operate the Kolmorov-Smirnov test in an attempt to perform Monte Carlo filtering as to reduce the distance observed

In [ ]:
KS_l = []
for c in Distance_df:
    parameters_l=[]
    for p in parameters.keys():
        B_dist=parameters[p][Distance_df[c][Distance_df[c]<Distance_df[c].median()].index].sort_values()
        B = (B_dist.cumsum()/B_dist.sum()).values
        NB_dist=parameters[p][Distance_df[c][Distance_df[c]>Distance_df[c].median()].index].sort_values()
        NB = (NB_dist.cumsum()/NB_dist.sum()).values
        parameters_l.append(stats.ks_2samp(B,NB)[1])
    KS_series = pd.Series(parameters_l,index=parameters.keys())
    KS_l.append(KS_series)
Kolmorov_Smirnov_df = pd.concat(KS_l,axis=1)
Kolmorov_Smirnov_df.columns = Distance_df.columns

### Get the KS significant outputs

In [ ]:
Kolmorov_Smirnov_df[Kolmorov_Smirnov_df<0.1]